In [1]:
CREATE DATABASE SCOPED CREDENTIAL PasinchukStorageCredential
WITH
    IDENTITY = 'bigdataschool001',
    SECRET = '';

: Msg 15530, Level 16, State 1, Line 1
The credential with name "PasinchukStorageCredential" already exists.

Total execution time: 00:00:00.101

In [2]:
CREATE EXTERNAL DATA SOURCE PassinchukTripDataSource
WITH
(
    LOCATION = 'wasbs://pasinchukdata@bigdataschool001.blob.core.windows.net/',
    CREDENTIAL = PasinchukStorageCredential,
    TYPE = HADOOP
);

: Msg 46502, Level 16, State 1, Line 1
Type with name 'PassinchukTripDataSource' already exists.

Total execution time: 00:00:00.068

In [15]:
CREATE EXTERNAL FILE FORMAT TripDataFormat
WITH
(
    FORMAT_TYPE = DELIMITEDTEXT,
    FORMAT_OPTIONS
    (
        FIELD_TERMINATOR = ',',
        First_Row = 2,
        USE_TYPE_DEFAULT = FALSE
    )
);

Commands completed successfully.

Total execution time: 00:00:00.077

 TODO: implement data parsing

In [16]:
CREATE EXTERNAL TABLE [pasinchuk_schema].[tripdata_ext](
    [VendorID]               [int]       NULL,
    [tpep_pickup_datetime]   [datetime]  NULL,
    [tpep_dropoff_datetime]  [datetime]  NULL,
    [passenger_count]        [int]       NULL,
    [trip_distance]          [real]      NULL,
    [RateCodeID]             [int]       NULL,
    [store_and_fwd_flag]     [char](1)   NULL,
    [PULocationID]           [int]       NULL,
    [DOLocationID]           [int]       NULL,
    [payment_type]           [int]       NULL,
    [fare_amount]            [real]      NULL,
    [extra]                  [real]      NULL,
    [mta_tax]                [real]      NULL,
    [tip_amount]             [real]      NULL,
    [tolls_amount]           [real]      NULL,
    [improvement_surcharge]  [real]      NULL,
    [total_amount]           [real]      NULL,
    [congestion_surcharge]   [real]      NULL
)
WITH (LOCATION='yellow_tripdata_2020-01.csv',
    DATA_SOURCE = PassinchukTripDataSource,  
    FILE_FORMAT = TripDataFormat,
    REJECT_TYPE = VALUE,
    REJECT_VALUE = 0
);  

Commands completed successfully.

Total execution time: 00:00:00.338

In [17]:
CREATE TABLE pasinchuk_schema.fact_tripdata
WITH (CLUSTERED COLUMNSTORE INDEX, DISTRIBUTION=HASH(tpep_dropoff_datetime))
AS SELECT * FROM pasinchuk_schema.tripdata_ext


Commands completed successfully.

Total execution time: 00:01:16.581

In [1]:
CREATE TABLE pasinchuk_schema.vendor_data (
    [VendorID]               [int]           NOT NULL,
    [Name]                   [nvarchar](50)  NOT NULL
);

: Msg 2714, Level 16, State 1, Line 1
There is already an object named 'vendor_data' in the database.

Total execution time: 00:00:00.105

In [6]:
INSERT INTO pasinchuk_schema.vendor_data ([VendorID] , [Name]) VALUES (1, 'Creative Mobile Technologies');
INSERT INTO pasinchuk_schema.vendor_data ([VendorID] , [Name]) VALUES (2, 'VeriFone Inc.');

(1 row affected)

Total execution time: 00:00:00.121

In [8]:
CREATE TABLE pasinchuk_schema.rate_data (
    [RateCodeID]             [int]           NOT NULL,
    [Name]                   [nvarchar](50)  NOT NULL
);

Commands completed successfully.

Total execution time: 00:00:00.830

In [9]:
INSERT INTO pasinchuk_schema.rate_data ([RateCodeID] , [Name]) VALUES (1, 'Standard rate');
INSERT INTO pasinchuk_schema.rate_data ([RateCodeID] , [Name]) VALUES (2, 'JFK');
INSERT INTO pasinchuk_schema.rate_data ([RateCodeID] , [Name]) VALUES (3, 'Newark');
INSERT INTO pasinchuk_schema.rate_data ([RateCodeID] , [Name]) VALUES (4, 'Nassau or Westchester');
INSERT INTO pasinchuk_schema.rate_data ([RateCodeID] , [Name]) VALUES (5, 'Negotiated fare');
INSERT INTO pasinchuk_schema.rate_data ([RateCodeID] , [Name]) VALUES (6, 'Group ride');

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.423

In [10]:
CREATE TABLE pasinchuk_schema.payment_data (
    [payment_type]           [int]           NOT NULL,
    [Name]                   [nvarchar](50)  NOT NULL
);

Commands completed successfully.

Total execution time: 00:00:00.643

In [11]:
INSERT INTO pasinchuk_schema.payment_data ([payment_type] , [Name]) VALUES (1, 'Credit card');
INSERT INTO pasinchuk_schema.payment_data ([payment_type] , [Name]) VALUES (2, 'Cash');
INSERT INTO pasinchuk_schema.payment_data ([payment_type] , [Name]) VALUES (3, 'No charge');
INSERT INTO pasinchuk_schema.payment_data ([payment_type] , [Name]) VALUES (4, 'Dispute');
INSERT INTO pasinchuk_schema.payment_data ([payment_type] , [Name]) VALUES (5, 'Unknown');
INSERT INTO pasinchuk_schema.payment_data ([payment_type] , [Name]) VALUES (6, 'Voided trip');

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.417